<a href="https://colab.research.google.com/github/SamratChakraborty99/Python/blob/main/Demorun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from urllib.parse import quote

#------------------------------------------------------------------------------------------------------#
                            # This part is used to fetch the cnsx symbol #

def Fetch_Company_Name(eq):
  url='https://www.google.com/search?q=site:Bloomberg.com '+eq.split()[0]+':IN'
  res=requests.get(url)
  name=None
  if res.status_code==200:
    soup=BeautifulSoup(res.content, 'html.parser')
    links=[m.text for m in soup.find_all('a')]
    for j in links:
      if j.endswith(f'{eq.split()[0]}:IN') and 'Ltd' in j:
        name=j.split('Ltd')[0]+"Ltd"
      if not name and j.endswith(f'{eq.split()[0]}:IN') and 'Stock Price Quote' in j:
        name=j.split(' Stock Price Quote')[0]
      if name and  ': ' in name:
        name=name.split(': ')[1]
  return name

def Fetch_original_name(name):
  url='https://www.google.com/search?q=site%3Awww.google.com%2Ffinance+'+f"{quote(name)}"
  res=requests.get(url)
  if res.status_code==200:
    soup=BeautifulSoup(res.content, 'html.parser')
    links=[m.text for m in soup.find_all('div', {'class':'BNeawe UPmit AP7Wnd lRVwie'})]
    for i in links:
      if i.endswith(':NSE'):
        return(i.split()[-1].split(':')[0])

def Fetch_CNSX(eq):
  name=Fetch_Company_Name(eq)
  url=f'http://www.google.com/finance?&q={name}'
  headers={'User-Agent': 'Mozilla/5.0'}
  resp=requests.get(url,headers=headers)
  title=BeautifulSoup(resp.content, 'html.parser')
  try:
    title=title.find('div',{'class':'PdOqHc'}).contents[-1].split(' • NSE')[0]
  except:
    title=Fetch_original_name(name)
  return title+'.NS'

def Fetch_CNSX_List(x):
  found=[]
  c=0
  try:
    for i in x:
      v=Fetch_CNSX(i)
      c+=1
      found.append((i,v))
      print(f"{i} ---> {v}")
    print("List of Symbols")
    print(found)
  except:
    print("Here is the list till which the names are found")
    print(found)
    print("here is the Remaining list")
    print(x[c:])
    print("PLEASE DISCONNECT AND RE-CONNECT THE COLLAB NOTEBOOK AND PROCESS THE REST.")

#--------------------------------------------------------------------------------------------------------------------#
                                  # Thsi part is used to download the data #

def get_date(d):
    date_object=datetime.strptime(d, '%Y-%m-%d')
    while date_object.strftime('%A')=='Saturday' or date_object.strftime('%A')=='Sunday':
        date_object-=timedelta(days=1)
    return date_object.strftime('%Y-%m-%d')

def stocks_for_each_year(stock_name,f,t):
    data_list=[]
    for i in range(f,t+1):
        date=f'{i}-12-31'
        date=get_date(date)
        date_object = datetime.strptime(date, '%Y-%m-%d')+ timedelta(days=1)
        new_date= date_object.strftime('%Y-%m-%d')
        tickerData = yf.Ticker(stock_name)
        data = tickerData.history(start=date, end=new_date)
        data=pd.DataFrame(data)
        data_list.append(data)
    data_list=pd.concat(data_list)
    return data_list

def stocks_freq(stock_name,f,t,freq):
    data_list=[]
    for i in range(f,t+1):
        start_date=f'{i}-01-01'
        end_date=f'{i}-12-31'
        tickerData = yf.Ticker(stock_name)
        data = tickerData.history(start=start_date, end=end_date, interval=freq)
        data=pd.DataFrame(data)
        data_list.append(data)
    data_list=pd.concat(data_list)
    return data_list

def process_stock(stock,name, f, t, freq):
    if freq=='1Y':
        x = stocks_for_each_year(stock, f,t)
    else:
        x=stocks_freq(stock,f,t,freq)
    x.columns = pd.MultiIndex.from_product([[name], x.columns])
    print(name)
    return x

def display_dataframe(names_and_symbols, f, t, freq):
    names=[]
    symbols=[]
    freq_list, f, t=[freq for i in range (len(names_and_symbols))],[f for i in range (len(names_and_symbols))],[t for i in range (len(names_and_symbols))]
    for i in names_and_symbols:
      names.append(i[0])
      symbols.append(i[1])
    print("Processed:")
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(process_stock, symbols, names, f, t, freq_list))
    final_dataframe = pd.concat(results, axis=1, join='outer')
    final_dataframe.index = final_dataframe.index.tz_convert(None)
    final_dataframe.to_excel('All_IB_Equity.xlsx')
    print("A excel file named 'All_IB_Equity.xlsx'has been saved to local pc in the same directory.")
    return final_dataframe


In [3]:
names = ['UPLL IB Equity', 'UTCEM IB Equity', 'VEDL IB Equity', 'VOLT IB Equity', 'WPRO IB Equity', 'Z IB Equity']

Fetch_CNSX_List(names)

UPLL IB Equity ---> UPL.NS
UTCEM IB Equity ---> ULTRACEMCO.NS
VEDL IB Equity ---> VEDL.NS
VOLT IB Equity ---> VOLTAS.NS
WPRO IB Equity ---> WIPRO.NS
Z IB Equity ---> ZEEL.NS
List of Symbols
[('UPLL IB Equity', 'UPL.NS'), ('UTCEM IB Equity', 'ULTRACEMCO.NS'), ('VEDL IB Equity', 'VEDL.NS'), ('VOLT IB Equity', 'VOLTAS.NS'), ('WPRO IB Equity', 'WIPRO.NS'), ('Z IB Equity', 'ZEEL.NS')]


In [7]:
names_and_symbols=[('ACC IB Equity', 'ACC.NS'), ('ACEM IB Equity', 'AMBUJACEM.NS'), ('ADANI IB Equity', 'ADANIPOWER.NS'), ('ADSEZ IB Equity', 'ADANIPORTS.NS'), ('AL IB Equity', 'ASHOKLEY.NS'), ('APHS IB Equity', 'APOLLOHOSP.NS'), ('APNT IB Equity', 'ASIANPAINT.NS'), ('ARBP IB Equity', 'AUROPHARMA.NS'), ('AUBANK IB Equity', 'AUBANK.NS'), ('AXSB IB Equity', 'AXISBANK.NS'), ('BAF IB Equity', 'BAJFINANCE.NS'), ('BANDHAN IB Equity', 'BANDHANBNK.NS'), ('BHARTI IB Equity', 'BHARTIARTL.NS'), ('BHE IB Equity', 'BEL.NS'), ('BHFC IB Equity', 'BHARATFORG.NS'), ('BJAUT IB Equity', 'BAJAJ-AUTO.NS'), ('BJFIN IB Equity', 'BAJAJFINSV.NS'), ('BOB IB Equity', 'BANKBARODA.NS'), ('BPCL IB Equity', 'BPCL.NS'), ('BRIT IB Equity', 'BRITANNIA.NS'), ('CIPLA IB Equity', 'CIPLA.NS'), ('CLGT IB Equity', 'COLPAL.NS'), ('COAL IB Equity', 'COALINDIA.NS'), ('DABUR IB Equity', 'DABUR.NS'), ('DIVI IB Equity', 'DIVISLAB.NS'), ('DLFU IB Equity', 'DLF.NS'), ('DMART IB Equity', 'DMART.NS'), ('DRRD IB Equity', 'DRREDDY.NS'), ('EIM IB Equity', 'EICHERMOT.NS'), ('FB IB Equity', 'FEDERALBNK.NS'), ('GAIL IB Equity', 'GAIL.NS'), ('GCPL IB Equity', 'GODREJCP.NS'), ('GPL IB Equity', 'GODREJPROP.NS'), ('GRASIM IB Equity', 'GRASIM.NS'), ('HAVL IB Equity', 'HAVELLS.NS'), ('HCLT IB Equity', 'HCLTECH.NS'), ('HDFCB IB Equity', 'HDFCBANK.NS'), ('HMCL IB Equity', 'HEROMOTOCO.NS'), ('HNDL IB Equity', 'HINDALCO.NS'), ('HUVR IB Equity', 'HINDUNILVR.NS'), ('ICICIBC IB Equity', 'ICICIBANK.NS'), ('ICICIGI IB Equity', 'ICICIGI.NS'), ('IIB IB Equity', 'INDUSINDBK.NS'), ('INDIGO IB Equity', 'INDIGO.NS'), ('INDUSTOW IB Equity', 'INDUSTOWER.NS'), ('INFO IB Equity', 'INFY.NS'), ('INFOE IB Equity', 'NAUKRI.NS'), ('IOCL IB Equity', 'IOC.NS'), ('IPRU IB Equity', 'ICICIPRULI.NS'), ('IRCTC IB Equity', 'IRCTC.NS'), ('ITC IB Equity', 'ITC.NS'), ('JSTL IB Equity', 'JSWSTEEL.NS'), ('JUBI IB Equity', 'JUBLFOOD.NS'), ('KMB IB Equity', 'KOTAKBANK.NS'), ('LPC IB Equity', 'LUPIN.NS'), ('LT IB Equity', 'LT.NS'), ('LTIM IB Equity', 'LTIM.NS'), ('MAXF IB Equity', 'MFSL.NS'), ('MM IB Equity', 'M&M.NS'), ('MPHL IB Equity', 'MPHASIS.NS'), ('MRCO IB Equity', 'MARICO.NS'), ('MSIL IB Equity', 'MARUTI.NS'), ('NEST IB Equity', 'NESTLEIND.NS'), ('NTPC IB Equity', 'NTPC.NS'), ('ONGC IB Equity', 'ONGC.NS'), ('PAG IB Equity', 'PAGEIND.NS'), ('PI IB Equity', 'PIIND.NS'), ('PIDI IB Equity', 'PIDILITIND.NS'), ('PWGR IB Equity', 'POWERGRID.NS'), ('RIL IB Equity', 'RELIANCE.NS'), ('SBICARD IB Equity', 'SBICARD.NS'), ('SBILIFE IB Equity', 'SBILIFE.NS'), ('SBIN IB Equity', 'SBIN.NS'), ('SIEM IB Equity', 'SIEMENS.NS'), ('SRCM IB Equity', 'SHREECEM.NS'), ('SRF IB Equity', 'SRF.NS'), ('SUNP IB Equity', 'SUNPHARMA.NS'), ('TATA IB Equity', 'TATASTEEL.NS'), ('TATACONS IB Equity', 'TATACONSUM.NS'), ('TCS IB Equity', 'TCS.NS'), ('TECHM IB Equity', 'TECHM.NS'), ('TPWR IB Equity', 'TATAPOWER.NS'), ('TRENT IB Equity', 'TRENT.NS'), ('TTAN IB Equity', 'TITAN.NS'), ('TTMT IB Equity', 'TATAMOTORS.NS'), ('UPLL IB Equity', 'UPL.NS'), ('UTCEM IB Equity', 'ULTRACEMCO.NS'), ('VEDL IB Equity', 'VEDL.NS'), ('VOLT IB Equity', 'VOLTAS.NS'), ('WPRO IB Equity', 'WIPRO.NS'), ('Z IB Equity', 'ZEEL.NS')]

display_dataframe(names_and_symbols,2007,2023,'1mo')

Processed:


ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1167589800, endDate = 1199039400
ERROR:yfinance:ADANIPOWER.NS: Data doesn't exist for startDate = 1167589800, endDate = 1199039400
ERROR:yfinance:ADANIPOWER.NS: Data doesn't exist for startDate = 1199125800, endDate = 1230661800
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1199125800, endDate = 1230661800
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1230748200, endDate = 1262197800
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1262284200, endDate = 1293733800
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1293820200, endDate = 1325269800
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1325356200, endDate = 1356892200
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1356978600, endDate = 1388428200
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1388514600, endDate = 1419964200
ERROR:yfinance:AUBANK.NS: Data doesn't exist for sta

ADANI IB Equity
ACEM IB EquityAUBANK IB Equity

ADSEZ IB Equity
APHS IB Equity
AXSB IB Equity


ERROR:yfinance:BANDHANBNK.NS: Data doesn't exist for startDate = 1167589800, endDate = 1199039400


ACC IB Equity
AL IB Equity
APNT IB Equity
ARBP IB Equity


...

ERROR:yfinance:SBICARD.NS: Data doesn't exist for startDate = 1325356200, endDate = 1356892200
ERROR:yfinance:SBILIFE.NS: Data doesn't exist for startDate = 1325356200, endDate = 1356892200
ERROR:yfinance:SBICARD.NS: Data doesn't exist for startDate = 1356978600, endDate = 1388428200
ERROR:yfinance:SBILIFE.NS: Data doesn't exist for startDate = 1356978600, endDate = 1388428200
ERROR:yfinance:SBILIFE.NS: Data doesn't exist for startDate = 1388514600, endDate = 1419964200
ERROR:yfinance:SBICARD.NS: Data doesn't exist for startDate = 1388514600, endDate = 1419964200
ERROR:yfinance:SBICARD.NS: Data doesn't exist for startDate = 1420050600, endDate = 1451500200
ERROR:yfinance:SBILIFE.NS: Data doesn't exist for startDate = 1420050600, endDate = 1451500200
ERROR:yfinance:SBICARD.NS: Data doesn't exist for startDate = 1451586600, endDate = 1483122600
ERROR:yfinance:SBILIFE.NS: Data doesn't exist for startDate = 1451586600, endDate = 1483122600
ERROR:yfinance:SBICARD.NS: Data doesn't exist for 

PWGR IB Equity


ERROR:yfinance:SBICARD.NS: Data doesn't exist for startDate = 1546281000, endDate = 1577730600


RIL IB Equity
SBILIFE IB Equity
SBICARD IB Equity
SBIN IB Equity
SIEM IB Equity
SRF IB Equity
SUNP IB Equity
SRCM IB Equity
TATA IB Equity
TATACONS IB Equity
TECHM IB Equity
TCS IB Equity
TPWR IB Equity
TTMT IB Equity
TRENT IB Equity
UTCEM IB Equity
TTAN IB Equity
UPLL IB Equity
VEDL IB Equity
VOLT IB Equity
WPRO IB Equity
Z IB Equity
A excel file named 'All_IB_Equity.xlsx'has been saved to local pc in the same directory.


ACC IB Equity                                         \
                             Open         High          Low        Close   
Date                                                                       
2007-01-01 18:30:00    798.694298   799.350178   789.293590   796.471619   
2007-01-02 18:30:00    787.034593   804.815763   777.561028   782.370667   
2007-01-03 18:30:00    789.584947   794.176016   775.374604   781.058777   
2007-01-04 18:30:00    787.034521   787.034521   761.601629   768.597473   
2007-01-07 18:30:00    768.816057   772.423262   743.346739   749.613892   
...                           ...          ...          ...          ...   
2023-12-14 18:30:00   2239.949951  2241.500000  2206.000000  2212.350098   
2023-12-17 18:30:00   2232.000000  2240.800049  2211.000000  2237.250000   
2023-12-18 18:30:00   2238.000000  2254.000000  2187.699951  2202.750000   
2023-12-19 18:30:00   2209.000000  2217.050049  2070.000000  2086.649902   
2023-12-20 18:30:00   2055.000000  2115.000000  2046.050049  2107.399902   

                                                     ACEM IB Equity  \
                       Volume Dividends Stock Splits           Open   
Date                                                                  
2007-01-01 18:30:00  261173.0       0.0          0.0     108.642695   
2007-01-02 18:30:00  429012.0       0.0          0.0     104.711532   
2007-01-03 18:30:00  448277.0       0.0          0.0     105.068895   
2007-01-04 18:30:00  319430.0       0.0          0.0     101.852522   
2007-01-07 18:30:00  311429.0       0.0          0.0      99.922695   
...                       ...       ...          ...            ...   
2023-12-14 18:30:00  362643.0       0.0          0.0     523.700012   
2023-12-17 18:30:00  312895.0       0.0          0.0     530.000000   
2023-12-18 18:30:00  445489.0       0.0          0.0     528.400024   
2023-12-19 18:30:00  895756.0       0.0          0.0     524.000000   
2023-12-20 18:30:00  505419.0       0.0          0.0     481.000000   

                                             ... WPRO IB Equity            \
                           High         Low  ...         Volume Dividends   
Date                                         ...                            
2007-01-01 18:30:00  108.642695  100.780395  ...      2135195.0       0.0   
2007-01-02 18:30:00  105.319078  102.960386  ...      4827355.0       0.0   
2007-01-03 18:30:00  105.068895  101.566603  ...      3837519.0       0.0   
2007-01-04 18:30:00  102.888914   98.314490  ...      2771786.0       0.0   
2007-01-07 18:30:00   99.922695   97.206625  ...      3708888.0       0.0   
...                         ...         ...  ...            ...       ...   
2023-12-14 18:30:00  529.400024  514.500000  ...     20783617.0       0.0   
2023-12-17 18:30:00  532.500000  524.299988  ...     10091036.0       0.0   
2023-12-18 18:30:00  530.549988  517.000000  ...      8032347.0       0.0   
2023-12-19 18:30:00  526.299988  486.000000  ...     13224971.0       0.0   
2023-12-20 18:30:00  505.899994  480.450012  ...      5329764.0       0.0   

                                 Z IB Equity                          \
                    Stock Splits        Open        High         Low   
Date                                                                   
2007-01-01 18:30:00          0.0   96.292228   96.949510   94.057461   
2007-01-02 18:30:00          0.0   94.714727   95.618491   92.710010   
2007-01-03 18:30:00          0.0   93.367302   96.505840   93.170119   
2007-01-04 18:30:00          0.0   94.320377   94.484701   92.019884   
2007-01-07 18:30:00          0.0   92.644292   93.991722   89.620779   
...                          ...         ...         ...         ...   
2023-12-14 18:30:00          0.0  282.899994  286.549988  273.000000   
2023-12-17 18:30:00          0.0  270.850006  282.000000  266.000000   
2023-12-18 18:30:00          0.0  273.899994  277.000000  267.149994   
2023-12-19 18:30:

# New Section